## Importing Different Packages

In [68]:
import numpy as np
import math, os
import pandas as pd
import json
import re
import random
from itertools import chain, combinations
import re
import numpy as np
import os
import random
import json
import string
from random import shuffle
import time
from sklearn.cluster import AgglomerativeClustering
import sys
import time
from sklearn.utils import shuffle
import sys


## Loading data

In [69]:
os.chdir("/Users/johanpoort/Documents/Studie algemeen/Econometrics/Paper")
with open('TVs-all-merged.json') as data:
    data_json = json.load(data)


r = 5 #set number of nrows in LSH

#Create dictionary
Data = dict(list(data_json.items())[:])

N = 1624
# print(len(Data))
# print(type(Data))





In [70]:
#split in 0.63 bootstrap data

lengthData=int(len(data_json.keys())*0.63) #63% of data for bootstrap



## Clean Data

In [71]:
"""In this section we will clean the data using several functions"""

# %% cleaning
def cl_unit(x):
    x = x.lower
    result = x.replace('"', "inch") and x.replace(' "', "inch") and x.replace('inches', 'inch') and x.replace(' inch','inch') and x.replace('-inch', 'inch') and x.replace('hertz', 'hz') and x.replace('-hz', 'hz') and x.replace(' hz','hz') and x.replace(' lbs', 'lbs')
    return result


def cl_title(x):
    x = x.translate(str.maketrans("", "", string.punctuation))
    x = x.replace('inchwatt', 'inch')
    x = ' '.join(w for w in x.split() if any(x.isdigit() for x in w))

    return x

def cl_title2(x):
    x = x.lower()
    x = ' '.join(w for w in x.split() if any(x for x in w))

    return x

def cl_value(x):
    x = x.lower()
    x = x.translate(str.maketrans("", "", string.punctuation))
    x = cl_unit(x)

    return x


def cl_shop(x):
    x = x.lower()
    x = x.translate(str.maketrans("", "", string.punctuation))
    return x


def cl_b(x):
    x = x.lower()
    # x.replace('pansonic','panasonic')
    # x.replace('lg electronics','lg')
    return x

## find model id

In [72]:
def findmodelID(x):
    x = x.replace('(', '')
    x = x.replace(')', '')
    x = x.replace('inch','')
    x = x.replace('Inch','')
    x = x.replace('720p','')
    x = x.replace('1080p','')
    x = x.replace('Hz','')
    x = x.replace('"','')


    temporary = ' '.join(word for word in x.split() if any(x.isdigit() for x in word)).split()
    print(temporary)
    wordlengthrd = ''
    for word in temporary:
        # TODO: if wordlengthrd contains number, charachter, number or character, number, character, do add. otherwise don't
        modelIDword = re.findall(r"\B[0-9A-Z]*\B", word)
        len1 = len(wordlengthrd)
        len2 = len(modelIDword)
        if len2 > len1:
            wordlengthrd = modelIDword
    return wordlengthrd
    modelID = []

    for word in temporary:
        if re.findall('[a-zA-Z0-9][0-9]+[a-zA-Z]+[0-9]+[a-zA-Z0-9]|[a-zA-Z0-9]*[a-zA-Z]+[0-9]+[a-zA-Z]+[a-zA-Z0-9]+', word):
            modelID.append(word)

    if len(modelID) > 1:
        longest_modelID = modelID[1]
        for ID in modelID:
            if len(ID) > len(longest_modelID):
                longest_modelID = ID
        modelID = [longest_modelID]
    return modelID

brandnames = {}
for key in Data.keys():
    for i in range(len(Data[key])):
        for k in Data[key][i]['featuresMap'].keys():
            if k == 'Brand' or k == 'Brand Name' or k == 'Brand Name:':
                if Data[key][i]['featuresMap'][k] not in brandnames.keys():
                    brandnames[Data[key][i]['featuresMap'][k]] = 1
                else:
                    brandnames[Data[key][i]['featuresMap'][k]] += 1

DListOfBrands = brandnames.keys()
ListOfBrands = [cl_b(item) for item in DListOfBrands]

# ListOfBrands = [cl_b(item) for item in brand_list]


In [73]:
"""Next, we are going to create 5 seperate dictionaries for key value pairs, potential model ID, title and shop"""

keys = []
titles = []
shop = []
keyvaluepairs = []
potmodelID = []
brand = []
titles2 = []

for key in Data.keys():
    for i in range(len(Data[key])):
        keys.append(key)
        potmodelID.append(findmodelID(Data[key][i]['title']))
        titles.append(cl_title(Data[key][i]['title']).split())
        shop.append(cl_shop(Data[key][i]['shop']))
        titles2.append(cl_title2(Data[key][i]['title']).split())

# print(keys)
# print((keys))
# # keys = dict(keys)
# print(keys)
# print(potmodelID)

keys_dict = {}
title_dict = {}
shop_dict = {}
potmodelID_dict = {}
brands_dict = {}
title2_dict = {}

for i in range(len(keys)):
    keys_dict.update({i: keys[i]})
    title_dict.update({i: titles[i]})
    shop_dict.update({i: shop[i]})
    potmodelID_dict.update({i: potmodelID[i]})
    title2_dict.update({i: titles2[i]})

# print(keys_dict)
# print(potmodelID_dict)
print(title_dict)
# print(shop_dict)
print(title2_dict)

# """We now have all the data in dictionaries. Next, we either want to create databases or keep on using the seperate dictionaries"""





['4000', '29', '2812', '60', '29PFL4508F7']
['32', 'SC-3211']
['90', '90', '120', '3D', 'LC-90LE657U']
['2000', '39', '3858', '60', '39PFL2908F7']
['70', '69.5', '120', 'LC70LE550U']
['40', '120', 'UN40F6350A']
['40', '40', 'HG40NA577LF']
['46', '46', 'UN46F5000AF']
['46', '46', 'E463']
['UN46ES6580', '46-', '120', '3D']
['46', '45.9', '240', 'UN46ES6580']
['39', '3812', '60', 'LE39FHDE3010']
['32', '60']
['32', '31.5', '60', '32SL410U']
['42', '60', 'LTDN42V68US']
['40', '60']
['40', '60', 'KDL40BX420']
['PN60E8000', '60-', '600', '3D', '2012']
['40', '40', 'H40B']
['26', '26', '60', '26LN4500']
['26', '26.0', '60', '26LN4500']
['55', '120', 'DP55441']
['70', '120', 'LC70LE650U']
['PN51E8000', '51-', '600', '3D', '2012']
['64']
['32', '31.5', '60', 'LEDTV3226']
['42', '600']
['42', '41.6', '600', 'TC-P42X3']
['XBR65HX950', '65', '3D']
['43', '43', '600', 'PN43F4500AFXZA']
['43', 'PN43F4500AFXZA']
['55LM8600', '55-', '3D', '240', '3D', '2012']
['55', '54.6', '3D', '240', '55LM8600']
['

In [74]:
"""Next, we will create a signature matrix to store the different binary values"""

"We first have to create count the different number of input words in the signature matrix. The signature matrix will take the following dimensions: S is an empty signature matrix of size (n, |P|)"


def findrbp(n, r):
    b = 1
    while r * b <= n * 1 / 2:
        b += 1
    p = r * b
    return p, r, b


"""Next, we are going to calculate the amount every word is used within the title and key value pairs
"""


# %% NWords
NWords = {}

for i in range(N):
    for word in title_dict[i]:
        if word not in NWords.keys():
            NWords[word] = 1
        else:
            NWords[word] += 1

"""Delete all words that are only used once in the NWords list"""
# print(NWords)
NWords = {k: v for k, v in NWords.items() if v >= 2}
print(NWords)
"""Create a list of all words that will be used as an input"""
input = list(NWords.keys())


"""We could now choose to add a certain value multiple times into the inputlist by using following code:"""
for i in range(3):
    for mid in potmodelID:
        input.append(mid)

    # %% creating input matrix
print('Number of model words used:  ' ,len(input))
print(NWords)






{'4000': 2, '29': 31, '2812': 4, '720p': 273, '60Hz': 397, '32': 199, '90': 2, '1080p': 1293, '120Hz': 384, '3D': 464, '2000': 2, '39': 44, '3858': 7, '70': 35, '695': 2, 'LC70LE550U': 2, '40': 117, '46': 126, 'UN46ES6580': 2, '46Inch': 20, '459': 14, '240Hz': 171, '3812': 12, '315': 46, '42': 120, '60Inch': 14, '600Hz': 165, '2012': 55, '26': 30, '26LN4500': 2, '55': 173, 'LC70LE650U': 5, '51Inch': 8, '64': 22, '416': 5, '65': 64, '43': 12, 'PN43F4500AFXZA': 2, '55LM8600': 2, '55Inch': 40, '546': 39, '7': 3, '4K': 19, '5458': 39, '55LA6200': 2, '50': 118, '49910': 5, '550': 3, '45910': 13, 'UN46EH5000FXZA': 2, '47': 102, '46910': 14, '47G2': 3, '47Inch': 14, '469': 14, '24': 46, '235': 4, '58Inch': 3, '51': 24, '5034': 8, '60': 154, 'LC60LE650U': 4, '6401': 3, '3112': 37, 'UN32F5500AFXZA': 2, '5500': 3, '120': 45, '47LN5700': 2, '41910': 4, '42LN5400': 2, '419': 6, '120hz': 6, '400': 2, 'SLED3280': 2, '4912': 20, '50L7300U': 3, '495': 12, '75': 10, '7412': 4, 'UN75F6400AFXZA': 2, '745

In [ ]:
"""Next, we will create the binary vector"""

binaryVectorMatrix = np.zeros((len(input), N))

for p in range(binaryVectorMatrix.shape[1]):
    title = title_dict[p]
    for word_i in range(binaryVectorMatrix.shape[0]):
        if input[word_i] in title:
            binaryVectorMatrix[word_i][p] = 1

del title, word_i

"""Next, we will use the binary values of 1 found modelID to match the binary values of another modelID. This way, """
for i in range(binaryVectorMatrix.shape[1]):
    for j in range(binaryVectorMatrix.shape[1]):
        if potmodelID_dict[i]==potmodelID_dict[j]:
            for word_i in range(binaryVectorMatrix.shape[0]):
                if binaryVectorMatrix[word_i][i] != 1:
                    binaryVectorMatrix[word_i][i] = binaryVectorMatrix[word_i][j]


print(binaryVectorMatrix)
print(binaryVectorMatrix.shape)



In [ ]:
"""Now we will create the signature matrix"""
r = 5
"""First we create a function to find a prime"""
def isPrime(x):
    for j in range(2, int(x ** 0.5) + 1):
        if x % j == 0:
            return False
    return True

def findPrimeNum(num):
    for i in range(num, 10000, 1):
        if isPrime(i):
            return i

"""We set the number of rows to 5 and select the number of adequate bands for a given n"""

(perm, nrows, nbands) = findrbp(N, r)
print(("(perm, nrows, nbands is )",perm, nrows, nbands))
print(N/3*2)

print('')
print('Number of perm    :  ', perm)
## we estimate the threshold by (1/b)^(1/r)
th = pow((1 / nbands), (1 / nrows))



"""We then create a signature matrix containing only millions"""
SignatureMatrix = np.ones((perm, N)) * 1000000
print(binaryVectorMatrix.shape[0])
a = random.sample(range(binaryVectorMatrix.shape[0]), perm)
b = random.sample(range(binaryVectorMatrix.shape[0]), perm)

print(a)
print(b)

print(SignatureMatrix)
print(SignatureMatrix.shape)

print(findPrimeNum(perm))

def hashing(a, b, r):
    return (int(a * r + b)) % findPrimeNum(perm)


def hashing2(n):
    return lambda x: hashing(a[n], b[n], x)


hashes = [hashing2(i) for i in range(perm)]
print(hashes[1](1))
#
for r in range(binaryVectorMatrix.shape[0]): #binaryVectorMatrix.shape[0] has as length the number of input
    hashr = []
    for permutation in range(perm):
        hashr.append(hashes[permutation](r))
    for p in range(binaryVectorMatrix.shape[1]):
        if binaryVectorMatrix[r][p] == 1:
            for permutation in range(perm):
                SignatureMatrix[permutation][p] = hashr[permutation]
# print(hashr)
print(SignatureMatrix)

In [ ]:
reps = 10
buckets = {}
for r in range(reps):
    shuff = SignatureMatrix.copy()
    np.random.shuffle(shuff)
    for p in range(shuff.shape[1]):
        for b in range(nbands):
            h = str(reps) + ' ' + str(b) + ' ' + (str([round(item) for item in shuff[:, p][b:b + nrows]]))
            if h not in buckets.keys():
                buckets[h] = [p]
            else:
                if p not in buckets[h]:
                    buckets[h].append(p)

buckets = {k: v for k, v in buckets.items() if len(v) >= 2}

del reps, p, b, h, shuff, r

"""We will start of by looking at all possible pairs"""
listoftruepairs = []
for i in range(N):
    for j in range(N):
        if i >= j:
            continue
        else:
            if keys_dict[i] == keys_dict[j]:
                pair = tuple([i, j])
                listoftruepairs.append(pair)

"""We then create a list of combinations of the different products in the same bucket"""
candidates = {}
listbucketcombinations = []
for key in buckets.keys():
    for comb in list(combinations(buckets[key], 2)):
        # listbucketcombinations.append(combinations(buckets[key], 2))
        if comb[0] > comb[1]:
            comb = tuple([comb[1], comb[0]])
        if comb not in candidates.keys():
            candidates[comb] = 1
        else:
            candidates[comb] += 1
# print(comb)
del key, comb


# %% remove same shop and different brand
begin = len(candidates.keys())
del_keys = []
for key in candidates.keys():
    i = key[0]
    j = key[1]
    if shop_dict[i] == shop_dict[j]:
        del_keys.append(key)

for key in del_keys:
    del candidates[key]
end = len(candidates.keys())


# %%Brand Search
begin = len(candidates.keys())

television_b = {}
for i in range(N):
    television_b[i] = 'none'
    for b in ListOfBrands:
        for blist in range(len(title2_dict[i])):
            if b in title2_dict[i][blist]:
                television_b[i] = b
                break

television_withb = {k: v for k, v in television_b.items() if v != 'none'}

# %%
del_keys = []
for key in candidates.keys():
    b_i = television_b[key[0]]
    b_j = television_b[key[1]]
    if b_i == 'none' or b_j == 'none':
        continue
    else:
        if b_i != b_j:
            del_keys.append(key)

for key in del_keys:
    del candidates[key]
end = len(candidates.keys())

print("th =",th)

In [ ]:
# print(title2_dict[i][1])
# print(range(len(title2_dict[1])))

In [ ]:
# print(television_brand)
print(ListOfBrands)

## Evaluation

## Clustering

In [ ]:
possible = {}

"""Calculate the number of true positives"""
true_pos = 0
x = []
for candidate in list(candidates.keys()):
    if keys_dict[candidate[0]] == keys_dict[candidate[1]]:
        true_pos += 1
        x.append(keys_dict[candidate[0]])
    else:
        continue

comb_total = sum(np.arange(0, len(Data) + 1, 1).tolist())
PQ = true_pos / len(candidates)
PC = true_pos / len(listoftruepairs)
F1 = (2 * PQ * PC) / (PQ + PC)

# print("number of rows =",r)
print('PQ = ',PQ)
print('PC = ',PC)
print('F1* = ',F1)



In [ ]:

# %% CLustering:
from sklearn.cluster import AgglomerativeClustering


def jacsim(a, b):
    v1 = SignatureMatrix[:, a]
    v2 = SignatureMatrix[:, b]
    v3 = v1 - v2
    v3[v3 != 0] = 1
    jdis = sum(v3) / len(v3)
    # if tuple([a,b]) in  potmodelIDpairs:
    #     return 1 - 0.5*(0+jdis)
    return 1 - jdis


DistanceMatrix = np.ones((len(keys_dict), len(keys_dict))) * 10000000
for i in range(DistanceMatrix.shape[0]):
    for j in range(DistanceMatrix.shape[1]):
        if tuple([i, j]) in candidates.keys():
            DistanceMatrix[i][j] = 1 - jacsim(i, j)

clustering = AgglomerativeClustering(affinity='precomputed', linkage='single', distance_threshold=0.25,
                                     n_clusters=None).fit_predict(DistanceMatrix)

selectBucket = {}
for i in range(len(clustering)):
    if clustering[i] not in selectBucket.keys():
        selectBucket[clustering[i]] = [i]
    else:
        selectBucket[clustering[i]].append(i)
selectBucket = {k: v for k, v in selectBucket.items() if len(v) >= 2}
result = []
for key in selectBucket.keys():
    for comb in list(combinations(selectBucket[key], 2)):
        if comb[0] > comb[1]:
            comb = tuple([comb[1], comb[0]])
        result.append(comb)